In [13]:
import torch
import os
import torch_pruning as tp
from models_training import ImageClassificationModel
from torchinfo import summary

In [14]:
# Initialize the model architecture
model = ImageClassificationModel(model_name='efficientnetv2s', num_classes=2)

In [15]:

# Load the state dictionary
state_dict = torch.load('state_dictionaries/.neptune/FIN-33/FIN-33/checkpoints/epoch=4-step=3910.pth')


In [16]:
# Retrieve the model name
model_name = state_dict.get('model_name', 'Unknown')
print(f'Loaded state dictionary for model: {model_name}')

Loaded state dictionary for model: efficientnetv2s


In [17]:
# Optionally, remove the model_name entry from the state_dict if needed
state_dict.pop('model_name', None)

'efficientnetv2s'

In [18]:

# Load the state dictionary into the model
model.load_state_dict(state_dict)


<All keys matched successfully>

In [19]:
model.hparams.model_name

'efficientnetv2s'

In [20]:
summary(model.model, input_size=(1, 3, 128, 128))

Layer (type:depth-idx)                                  Output Shape              Param #
EfficientNet                                            [1, 2]                    --
├─Sequential: 1-1                                       [1, 1280, 4, 4]           --
│    └─Conv2dNormActivation: 2-1                        [1, 24, 64, 64]           --
│    │    └─Conv2d: 3-1                                 [1, 24, 64, 64]           648
│    │    └─BatchNorm2d: 3-2                            [1, 24, 64, 64]           48
│    │    └─SiLU: 3-3                                   [1, 24, 64, 64]           --
│    └─Sequential: 2-2                                  [1, 24, 64, 64]           --
│    │    └─FusedMBConv: 3-4                            [1, 24, 64, 64]           5,232
│    │    └─FusedMBConv: 3-5                            [1, 24, 64, 64]           5,232
│    └─Sequential: 2-3                                  [1, 48, 32, 32]           --
│    │    └─FusedMBConv: 3-6                         

In [22]:
device = torch.device('cuda:0')

In [23]:
example_inputs = torch.randn(1, 3, 128, 128, device='cuda')

In [24]:
# Test forward in eval mode
print("====== Forward (Inferece with torch.no_grad) ======")
with torch.no_grad():
    laterncy_mu, latency_std= tp.utils.benchmark.measure_latency(model, example_inputs, repeat=300)
    print('latency: {:.4f} +/- {:.4f} ms'.format(laterncy_mu, latency_std))

    memory = tp.utils.benchmark.measure_memory(model, example_inputs, device=device)
    print('memory: {:.4f} MB'.format(memory/ (1024)**2))

    example_inputs_bs1 = torch.randn(1, 3, 128, 128, device='cuda')
    fps = tp.utils.benchmark.measure_fps(model, example_inputs_bs1, repeat=300)
    print('fps: {:.4f}'.format(fps))

    example_inputs = torch.randn(128, 3, 128, 128, device='cuda')
    throughput = tp.utils.benchmark.measure_throughput(model, example_inputs, repeat=300)
    print('throughput (bz=256): {:.4f} images/s'.format(throughput))

====== Forward (Inferece with torch.no_grad) ======
latency: 21.1618 +/- 5.4765 ms
memory: 372.2427 MB
fps: 47.8205
throughput (bz=256): 1295.9499 images/s


In [1]:
import os

In [5]:
from models_training import ImageClassificationModel

In [2]:
checkpoint_path = os.path.normpath('.neptune\FIN-67\FIN-67\checkpoints\epoch=4-step=3910.ckpt')

In [3]:
checkpoint_path

'.neptune\\FIN-67\\FIN-67\\checkpoints\\epoch=4-step=3910.ckpt'

In [6]:
model = ImageClassificationModel.load_from_checkpoint(checkpoint_path)

In [7]:
state_dict = model.state_dict()

In [8]:
# Include the model name in the state dictionary
state_dict['model_name'] = model.hparams.model_name

# Extract the necessary parts from the checkpoint path
base_name = os.path.basename(checkpoint_path)
dir_name = os.path.dirname(checkpoint_path)
save_dir = os.path.join('state_dictionaries', dir_name)

# Ensure the save directory exists
os.makedirs(save_dir, exist_ok=True)

In [10]:
import torch

In [11]:
# Create the save path for the state dictionary
save_path = os.path.join(save_dir, base_name.replace('.ckpt', '.pth'))

# Save the state dictionary
torch.save(state_dict, save_path)
print(f'Saved state dictionary at {save_path}')

Saved state dictionary at state_dictionaries\.neptune\FIN-67\FIN-67\checkpoints\epoch=4-step=3910.pth


In [12]:
import os

In [13]:
path = "state_dictionaries\.neptune\FIN-67\FIN-67\checkpoints\epoch=4-step=3910.pth"

In [14]:
test = os.path.normpath(path)

In [15]:
test

'state_dictionaries\\.neptune\\FIN-67\\FIN-67\\checkpoints\\epoch=4-step=3910.pth'

In [16]:
# Split the path based on the separator (\ in this case)
parts = test.split('\\')

# Find the index where 'state_dictionaries' is located
index = parts.index('state_dictionaries')

In [17]:
parts

['state_dictionaries',
 '.neptune',
 'FIN-67',
 'FIN-67',
 'checkpoints',
 'epoch=4-step=3910.pth']

In [9]:
# Extract the prefix path
prefix_path = os.path.join(*parts[:index + 3])

In [10]:
prefix_path

'state_dictionaries\\.neptune\\FIN-67'